# API work

FastAPI

## Chaetbot API part
- imput we will receive the **user_id**, **thread_id** and the initial **message**
    - output we will send the message to the user and the **thread_id**
Here we will use cahtconsierge assistant asst_CjvyFIeraCLKB8NTAqF0FhqG

## Sumariser part
- input we will receive the **message** and the **thread_id**
    - output we will send the **summary** and the **thread_id**

here we will use the summariser assistant asst_kCSrKaHjh589gbKr2fphQ93T

### Summariser

In [2]:
# Connect to openai 

import openai
import os
import time

openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()
assistant = client.beta.assistants.retrieve("asst_kCSrKaHjh589gbKr2fphQ93T")


# Open the file and read its contents
with open("message_thread.txt", 'r', encoding='utf-8') as file:
    message_to_send = file.read()
    
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=message_to_send
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def check_run_status(client, thread_id, run_id):
    try:
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        return run.status  # Access the status attribute directly
    except Exception as e:
        print(f"Error retrieving run: {e}")
        return None

def wait_for_run_completion(client, thread_id, run_id, interval=0.5):
    while True:
        status = check_run_status(client, thread_id, run_id)
        if status == 'completed':
            print("Run is completed.")
            break
        elif status in ['failed', 'cancelled']:
            print(f"Run {status}.")
            break
        else:
            print(f"Run is still in progress... Current status: {status}")
            time.sleep(interval)


# Get the thread ID and run ID
thread_id = thread.id
run_id = run.id

print(f"Thread ID: {thread_id}")
print(f"Run ID: {run_id}")
# print the separator
print("-" * 20)

# Wait for the run to complete
wait_for_run_completion(client, thread_id, run_id)

# print the separator
print("-" * 20)

messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# Variable to store the last assistant's response
last_assistant_response = ""

for message in messages.data:
    # Check if the message is from the assistant and has content
    if message.role == "assistant" and message.content:
        # Assuming the first item in content is a text object
        text_content = message.content[0].text
        if text_content:
            # Print and store the message text value
            last_assistant_response = text_content.value
            print(f"Assistant: {last_assistant_response}")  # Print the value

FileNotFoundError: [Errno 2] No such file or directory: 'message_thread.txt'

### Chatbot

In [1]:
# Connect to openai 

import openai
import os
import time

openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()
assistant = client.beta.assistants.retrieve("asst_CjvyFIeraCLKB8NTAqF0FhqG")

def send_message_get_reply(message, thread_id=None):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    client = openai.Client()
    assistant_id = "asst_CjvyFIeraCLKB8NTAqF0FhqG"

    # Создание нового треда, если thread_id не предоставлен
    if thread_id is None:
        thread = client.beta.threads.create()
        thread_id = thread.id

    # Отправка сообщения
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message
    )

    # Запуск обработки сообщения
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    # Проверка статуса выполнения
    def check_run_status(run_id):
        try:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            return run.status
        except Exception as e:
            print(f"Error retrieving run: {e}")
            return None

    # Ожидание ответа
    while True:
        status = check_run_status(run.id)
        print(f"Run is still in progress... Current status: {status}")
        if status == 'completed':
            break
        elif status in ['failed', 'cancelled']:
            return None, thread_id
        time.sleep(0.5)

    # Получение ответа
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = messages.data
    last_message = next(
        (m.content[0].text.value for m in messages if m.role == "assistant" and m.content),
        None
    )

    
    
    return last_message, thread_id

In [5]:
# Список сообщений для отправки
messages_to_send = [
    "Я хочу поехать в дубай",
    "Хочу забронировать тур",
    "Я поеду один",
    "Какие варианты трансфера существуют?",
    "Мне нужно будет заказать такси"
]

# Инициализация переменной для ID треда
thread_id = None

# Отправка сообщений и получение ответов
for message in messages_to_send:
    
    print(f"Сообщение, отправленное помощнику: {message}")
    response, thread_id = send_message_get_reply(message, thread_id)
    print(f"Ответ помощника: {response}, ID треда: {thread_id}")


Сообщение, отправленное помощнику: Я хочу поехать в дубай
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: completed
Ответ помощника: Отлично! Давайте уточним детали вашей поездки. Начнем с размещения. Вы ищете тур в Дубай или вам нужно только забронировать отель?, ID треда: thread_dRl9mWdwGGxYdO50tXPDUsOu
Сообщение, отправленное помощнику: Хочу забронировать тур
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: completed
Ответ помощника: Понял вас. На какое количество взрослых и детей вы планируете тур?, ID треда: thread_dRl9mWdwGGxYdO50tXPDUsOu
Сообщение,